In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import csv
import time

def scrape_tiki_powerbanks():
    # Cấu hình trình duyệt headless (chạy nền)
    options = Options()
    options.add_argument("--headless")  # Xoá dòng này nếu muốn xem trình duyệt chạy
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)  # Nếu bạn có chromedriver ở nơi khác, thêm: executable_path="..."

    # Truy cập trang tìm kiếm sạc dự phòng
    driver.get("https://tiki.vn/search?q=s%E1%BA%A1c%20d%E1%BB%B1%20ph%C3%B2ng")
    time.sleep(5)  # Chờ trang load

    # Cuộn xuống để load thêm sản phẩm
    for _ in range(3):  # cuộn 3 lần
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        time.sleep(2)

    # Tìm các thẻ sản phẩm
    products = driver.find_elements(By.CSS_SELECTOR, 'a[data-view-id="product_list_item"]')

    results = []
    for p in products:
        try:
            name = p.find_element(By.CSS_SELECTOR, 'div.title').text
            price = p.find_element(By.CSS_SELECTOR, 'div.price-discount__price').text
            image = p.find_element(By.TAG_NAME, 'img').get_attribute("src")
            url = p.get_attribute("href")
            results.append({
                "category": "Sạc dự phòng",
                "product_name": name,
                "price": price,
                "image": image,
                "source": url
            })
        except:
            continue

    driver.quit()

    # Lưu vào CSV
    with open("tiki_sac_du_phong_selenium.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["category", "product_name", "price", "image", "source"])
        writer.writeheader()
        writer.writerows(results)

    print(f"✅ Đã lưu {len(results)} sản phẩm từ Tiki vào tiki_sac_du_phong_selenium.csv")

# Gọi hàm
scrape_tiki_powerbanks()


🔎 Đang lấy trang 1 từ danh mục: Sạc dự phòng
🔎 Đang lấy trang 2 từ danh mục: Sạc dự phòng
🔎 Đang lấy trang 3 từ danh mục: Sạc dự phòng
✅ Đã lưu 0 sản phẩm vào tiki_sac_du_phong.csv
